# 動的計画法

## 導入

- 動的計画法とは
  - 最適化計算手法の一種
- 最適化計算とは
  - 関数の最大値or最小値を求める計算
  - 一般には全組合せの調査必要
- 関数が対ごとの2変数関数の和の場合
  - 最適解の効率的な解法 ＝ 動的計画法

## 方法

\begin{equation*}
J  = f_1(x_1) + h_1(x_1, x_2) + h_2(x_2, x_3) + \cdots + h_{n-1}(x_{n-1}, x_n) \rightarrow \max
\end{equation*}

- $x_1$に着目
- 第1項、第2項のみ
- 可能な全ての$x_2$に対し最適な$x_1$を計算
- 第1項、第2項は次式に書ける

\begin{equation*}
f_2(x_2)  = \max_{x_1}[f_1(x_1) + h_1(x_1, x_2)]
\end{equation*}

- 元の式を$f_2$を使って表現

\begin{equation*}
J  = f_2(x_2) + h_2(x_2, x_3) + \cdots + h_{n-1}(x_{n-1}, x_n) \rightarrow \max
\end{equation*}

- $x_2$, $x_3$と繰り返す
- $x_n$まで計算すると最適な$\{x_1, x_2, ..., x_n\}$が得られる

## カエル跳び問題

### 問題

$N$個の足場があって、$i$番目の足場の高さは$h_i$です。
最初、足場1にカエルがいて、ぴょんぴょん跳ねながら足場$N$へと向かいます。カエルは足場$i$にいるときに

- 足場$i$から足場$i+1$へと移動する (そのコストは$|h_i-h_{i+1}$)
- 足場$i$から足場$i+2$へと移動する (そのコストは$|h_i-h_{i+2}$)

のいずれかの行動を選べます。カエルが足場1から足場$N$へと移動するのに必要な最小コストを求めよ。

<img width="500" src="https://qiita-user-contents.imgix.net/https%3A%2F%2Fqiita-image-store.s3.amazonaws.com%2F0%2F182963%2Fc010da1b-e2d0-3121-9050-5d306dfe3c41.jpeg?ixlib=rb-4.0.0&auto=format&gif-q=60&q=75&w=1400&fit=max&s=1c8e3db3816ce58e329c8f08fbeb5a79">

In [2]:
import numpy as np
def Frog1(footholds):
    one_diffs = [abs(f0 - f1) for f0, f1 in zip(footholds, footholds[1:])]
    two_diffs = [abs(f0 - f2) for f0, f2 in zip(footholds, footholds[2:])]
    indexes = [[0]]
    scores = np.zeros((1,))
    indexes.append(indexes[-1] + [len(scores)])
    scores = np.append(scores, one_diffs.pop(0))
    for diff in zip(two_diffs, one_diffs):
        score = np.array([scores[-2] + diff[-2], scores[-1] + diff[-1]])
        i = np.argmin(score) - 2
        indexes.append(indexes[i] + [len(scores)])
        scores = np.append(scores, score[i])
    return indexes[-1]

In [13]:
footholds = [2, 9, 4, 5, 1, 6]
Frog1(footholds)

[0, 2, 3, 5]

## ナップサック問題

- $n$個の荷物
- 荷物$i$の重さ$w_i$, 価値$v_i$
- 重さの合計がUを越えない様に荷物を選ぶ
- 価値の合計の最大値を求めよ

In [18]:
def knapsack(weights, values, max_weight):
    num_things = len(weights)
    dp = [[0] * (max_weight + 1)] * (num_things + 1)
    indexes = [[[]] * (max_weight + 1)] * (num_things + 1)
    #
    for i, (weight, value) in enumerate(zip(weights, values)):
        for u in range(max_weight + 1):
            if u >= weight:
                if dp[i][u - weight] + value > dp[i][u]:
                    dp[i + 1][u] = dp[i][u - weight] + value
                    indexes[i + 1][u] = indexes[i][u] + [i]
                    continue
            dp[i + 1][u] = dp[i][u]
    return dp[num_things][max_weight], indexes[num_things][max_weight]

In [19]:
weights = [2, 1, 3, 2, 1, 5]
values = [3, 2, 6, 1, 3, 85]
max_weight = 9
knapsack(weights, values, max_weight)

(97, [0, 1, 4, 5])